# Symbiotic Relationships Simulation

In [18]:
from mesa.discrete_space import CellAgent, FixedAgent
import math
import mesa
from mesa import Model
from mesa.datacollection import DataCollector
from mesa.discrete_space import OrthogonalMooreGrid, CellAgent, FixedAgent
from mesa.experimental.devs import ABMSimulator

SEED = 42

class Animal(CellAgent):
    def __init__(self, model, cell=None):
        super().__init__(model)
        self.cell = cell

    def feed(self):
        """Abstract method to be implemented by subclasses."""
        
    def step(self):
        self.move()
    
    def move(self):
        self.cell=self.cell.neighborhood.select_random_cell()

        
class Frog(Animal):
        pass
class Spider(Animal):
        pass

class Ant(Animal):
        pass

class Snake(Animal):
        pass



In [19]:
class SymbioticRelationshipsModel(Model):
    def __init__(self, width=24, height=24, seed = None, rng = None, simulator: ABMSimulator = None):
        super().__init__(seed=seed, rng=rng)
        
        if simulator is None:
            simulator = ABMSimulator()

        self.simulator = simulator
        self.simulator.setup(self)
        
        self.height = height
        self.width = width
        
        # Create grid using experimental cell space
        self.grid = OrthogonalMooreGrid(
            [self.height, self.width],
            torus=False, # Decide upon this!?
            capacity=math.inf, #
            random=self.random,
        )
        
        # Set up data collection
        model_reporters = {
            "Spiders": lambda m: len(m.agents_by_type[Spider]),
            "Frogs": lambda m: len(m.agents_by_type[Frog]),
        }

        self.datacollector = DataCollector(model_reporters)
        
        
        Frog.create_agents(
            self,
            1, # frog amount
            cell=self.random.choices(self.grid.all_cells.cells),
        )

        Snake.create_agents(
            self,
            1, # frog amount
            cell=self.random.choices(self.grid.all_cells.cells),
        )
        
        Spider.create_agents(
            self,
            1, # frog amount
            cell=self.random.choices(self.grid.all_cells.cells),
        )

        Ant.create_agents(
            self,
            1, # frog amount
            cell=self.random.choices(self.grid.all_cells.cells),
        )
        
        # Collect initial data
        self.running = True
        self.datacollector.collect(self)
        
    def step(self):
        """Execute one step of the model."""
        self.agents_by_type[Frog].shuffle_do("step")
        self.agents_by_type[Spider].shuffle_do("step")
        self.agents_by_type[Ant].shuffle_do("step")
        self.agents_by_type[Snake].shuffle_do("step")
        
        # Collect data
        self.datacollector.collect(self)



In [20]:
from mesa.visualization import (
    CommandConsole,
    Slider,
    SolaraViz,
    make_plot_component,
    make_space_component,
)

# This function will be called for every agent present in the model and defines a visualization (color, marker...)
def agent_portrayal(agent):
    if agent is None:
        return

    portrayal = {
        # "size": 45,
    }

    # Filter the agent type, then describe the agent visuals
    if isinstance(agent, Frog):
        portrayal["color"] = "tab:green"
        portrayal["marker"] = "o"
        portrayal["zorder"] = 2

    elif isinstance(agent, Spider):
        portrayal["color"] = "tab:brown"
        portrayal["marker"] = "X"
        portrayal["zorder"] = 2

    elif isinstance(agent, Ant):
        portrayal["color"] = "tab:red"
        portrayal["marker"] = "d"
        portrayal["zorder"] = 2

    elif isinstance(agent, Snake):
        portrayal["color"] = "tab:orange"
        portrayal["marker"] = "s"
        portrayal["zorder"] = 2
    

    return portrayal


model_params = {
    "seed": {
        "type": "InputText",
        "value": SEED,
        "label": "Random Seed",
    }
}

def post_process_space(ax):
    ax.set_aspect("equal")
    ax.set_xticks([])
    ax.set_yticks([])

# The main grid with all the agents
space_component = make_space_component(
    agent_portrayal, draw_grid=True, post_process=post_process_space
)

def post_process_lines(ax):
    ax.legend(loc="center left", bbox_to_anchor=(1, 0.9))

simulator = ABMSimulator() 
model = SymbioticRelationshipsModel(seed=SEED, simulator=simulator)

lineplot_component = make_plot_component(
    {"Frogs": "tab:green", "Spiders": "tab:brown"},
    post_process=post_process_lines,
)

page = SolaraViz(
    model,
    # components=[space_component, lineplot_component, CommandConsole],
    components=[space_component, lineplot_component, CommandConsole],
    model_params=model_params,
    name="Symbiotic Relationships",
    simulator=simulator,
)

page  # noqa

Cannot show ipywidgets in text